<a href="https://colab.research.google.com/github/cgint/llama_index_t1/blob/main/LlamaIndex_Simple_Graph_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Graph LLM, Demo Outline

Got it from [Graph_RAG_LlamaIndex_Workshop.ipynb](https://colab.research.google.com/drive/1tLjOg2ZQuIClfuWrAC2LdiZHCov8oUbs#scrollTo=s5LPkzt1YUIN)




### Graph RAG

> Graph RAG with LLM

![Graph RAG](https://github.com/siwei-io/talks/assets/1651790/f783b592-7a8f-4eab-bd61-cf0837e83870)


> Query time workflow:

- Get Key Entities/Relationships related to task
  - LLM or NLP to extract from task string
  - Expand synonyms
- Get SubGraphs
  - Exact matching of "Starting Point"
  - Optionally Embedding based
- Generate answer based on SubGraphs
  - Could be combined with other RAG
  - If KG was built with LlamaIndex, metadata could be leveraged

> Values

- KG is __ of Knowledge:
  - Refined and Concise Form
  - Fine-grained Segmentation
  - Interconnected-structured nature
- Knowledge in (existing) KG is Accurate
- Query towards KG is Stable yet Deterministic
- Reasoning/Info. in KG persist domain knowledge

> Refs:

- https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html
- https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_rag_query_engine.html
- https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html
- https://siwei.io/talks/graph-rag-with-jerry/
- https://www.youtube.com/watch?v=bPoNCkjDmco

### KG Building

![KG Building](https://github.com/siwei-io/talks/assets/1651790/495e035e-7975-4b77-987a-26f8e1d763d2)

> Value

- Game-changer for ROI on adaptation of Graph
  - NLP Competence and efforts
  - Complex Pipelines
- Those "nice to have" graphs can now be enabled by Graph at a small cost

> Refs
- https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/NebulaGraphKGIndexDemo.html#instantiate-gptnebulagraph-kg-indexes
- https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_query_engine.html
- https://colab.research.google.com/drive/1G6pcR0pXvSkdMQlAK_P-IrYgo-_staxd?usp=sharing
- https://siwei.io/en/demos/text2cypher/
- https://siwei.io/demo-dumps/kg-llm/KG_Building.ipynb
- https://siwei.io/demo-dumps/kg-llm/KG_Building.html

# How

with LlamaIndex and Simple Graph in memory

## Concepts

REF: https://gpt-index.readthedocs.io/en/stable/getting_started/concepts.html

### RAG

Retrieval Augmented Generation:

![](https://gpt-index.readthedocs.io/en/stable/_images/rag.jpg)

### Indexing Stage

![](https://gpt-index.readthedocs.io/en/stable/_images/indexing.jpg)
- [Data Connectors(LlamaHub)](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/connector/root.html)

- Documents
  - Nodes(Chunk)
- Index
  - VectorIndex
  - [KnowledgeGraphIndex](https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html), create KG from data, Graph RAG
  - SQLIndex


### Querying Stage

- Query Engine/Chat Engine Agent(input text, output answer)
  - [KnowledgeGraphQueryEngine](https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_query_engine.html), Text2Cypher Query engine
- Retriever(input text, output nodes)
  - [KnowledgeGraphRAGRetriever](https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_rag_query_engine.html), for existing KG wired as Graph RAG
- Node Postprocessor(Reranking, filterring nodes)
- Response Synthesizer(input nodes, output answer)

![](https://gpt-index.readthedocs.io/en/stable/_images/querying.jpg)


### Context

REF:
- https://gpt-index.readthedocs.io/en/stable/core_modules/supporting_modules/service_context.html
- https://gpt-index.readthedocs.io/en/stable/api_reference/storage.html



Service context

- LLM
- Embedding Model
- Prompt Helper

Storage context

- Vector Store
- Graph Store





## Key KG related components

- [KnowledgeGraphIndex](https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphDemo.html) is an Index to:
  - Indexing stage:
    - Extract data into KG with LLM or any other callable models
    - Persist KG data into `storeage_context.graph_store`
  - Querying stage:
    - `as_query_engine()` to enable 0-shot Graph RAG
    - `as_retriever()` to create an advanced Graph involving RAG
- [KnowledgeGraphRAGRetriever](https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_rag_query_engine.html)
  - Instanctiate:
    - Create a `storeage_context.graph_store` as the init argument.
  - Querying stage:
    - pass to `RetrieverQueryEngine` to become a Graph RAG query engine on any existing KG
    - combined with other RAG pipeline

- [KnowledgeGraphQueryEngine](https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_query_engine.html), Text2Cypher Query engine
  - Instanctiate:
    - Create a `storeage_context.graph_store` as the init argument.
  - Querying stage:
    - Text2cypher to get answers towards the KG in graph_store.
    - Optionally, `generate_query()` only compose a cypher query.

## Preparation

Install Dependencies, prepare for contexts of Llama Index

In [1]:
%pip install openai llama_index pyvis

In [2]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "sk-..."

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

import logging
import sys

from IPython.display import Markdown, display


from llama_index.llms import OpenAI
from llama_index import set_global_service_context

# define LLM
llm = OpenAI(temperature=0, model="gpt-4-1106-preview") # gpt-3.5-turbo-1106
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

# set global service context
set_global_service_context(service_context)

## Create a Graph Space

KnowledgeGraphIndex on SimpleGraphStore


## Storage_context with Graph_Store

In [3]:
from llama_index.graph_stores import SimpleGraphStore
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 🏗️ KG Building with Llama Index

### Preprocess Data with data connectors

with `WikipediaReader`

We will download and preprecess data from:
    https://en.wikipedia.org/wiki/Guardians_of_the_Galaxy_Vol._3

In [4]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

### Indexing Extract Triplets and Save to KnowledgeGraph

with `KnowledgeGraphIndex`

This call will take some time, it'll extract entities and relationships and store them into KnowledgeGraph

In [5]:
print(f"About to build index over {len(documents)} document(s) ...")
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True
)

## 🧠 Graph RAG



### KG_Index as **Query Engine**

In [6]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [7]:
response_graph_rag = kg_index_query_engine.query("What challenges do Rocket and Lylla face?")

display(Markdown(f"<b>{response_graph_rag}</b>"))

Extracted keywords: ['Lylla', 'Rocket', 'challenges']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'critically wounded by', 'Adam Warlock']
['Adam Warlock', 'created by', 'High Priestess Ayesha']
['Adam Warlock', 'included in script', 'Vol. 2']
['Rocket', 'has kill switch embedded by', 'Orgocorp']
['Rocket', 'befriended', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'fled in', 'spaceship']
['Rocket', 'has', 'near-death experience']
['Rocket', 'reunites with', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'reunites with', 'Teefs']
['Rocket', 'reunites with', 'Floor']
['Rocket', 'could exist in', 'MCU']
['Rocket', 'is character in', 'Guardians of the Galaxy Vol. 3']
['Guardians of the Galaxy Vol. 3', 'is', 'film']
['Guardians o

<b>Rocket and Lylla face several challenges throughout their experiences. Rocket is critically wounded by Adam Warlock, which is a significant challenge as it puts his life in danger. Additionally, Rocket has a kill switch embedded in him by Orgocorp, which poses a threat to his life and requires the Guardians to find an override code to save him. Lylla, as an associate and friend of Rocket, is killed by the High Evolutionary, which is a traumatic event for Rocket, who is deeply affected by the loss of his friend. Rocket also faces the emotional challenge of being experimented on by the High Evolutionary and the pain of losing his fellow Batch 89 test subjects, Lylla, Teefs, and Floor. During a near-death experience, Rocket reunites with Lylla, Teefs, and Floor, and Lylla informs him that his time has not yet come, indicating that despite the challenges, Rocket still has a role to play.</b>

In [8]:
response_graph_rag = kg_index_query_engine.query("Tell me about James Gunn.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

Extracted keywords: ['James Gunn', 'James', 'Gunn']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['James Gunn', 'wrote and directed', 'Guardians of the Galaxy Vol. 3']
['Guardians of the Galaxy Vol. 3', 'is', 'film']
['Guardians of the Galaxy Vol. 3', 'released in', '2023']
['Guardians of the Galaxy Vol. 3', 'produced by', 'Marvel Studios']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy Vol. 2']
['Guardians of the Galaxy Vol. 3', 'is part of', 'Marvel Cinematic Universe']
['Guardians of the Galaxy Vol. 3', 'premiered at', 'Disneyland Paris']
['Guardians of the Galaxy Vol. 3', 'grossed over', '$845 million']
['Guardians of the Galaxy Vol. 3', 'set after', 'Ave

<b>James Gunn is a filmmaker who wrote and directed "Guardians of the Galaxy Vol. 3," which is a film released in 2023. He has been involved with the Marvel Cinematic Universe, particularly with the Guardians of the Galaxy franchise, having directed the previous two installments as well. His work on "Guardians of the Galaxy Vol. 3" was intended to conclude the story of the current iteration of the Guardians and to help launch new and existing Marvel characters into the future of the MCU. Despite facing a setback due to offensive statements discovered on his Twitter feed, which led to his temporary firing by Disney, Gunn was eventually rehired to continue his work on the third Guardians film. The film was produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. It is a sequel to "Guardians of the Galaxy" and "Guardians of the Galaxy Vol. 2," and it follows the events of "Avengers: Infinity War" and "Avengers: Endgame." The film premiered at Disneyland Paris and had an early screening at Dongdaemun Design Plaza. It was a commercial success, grossing over $845 million worldwide. James Gunn's involvement with the film also extended to curating its soundtrack, which features a diverse range of music spanning multiple decades.</b>

In [9]:
query_engine = kg_index_query_engine

question = """
Tell me about James Gunn.
"""

response_nl2kg = query_engine.query(question)

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

Extracted keywords: ['James Gunn', 'James', 'Gunn']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['James Gunn', 'wrote and directed', 'Guardians of the Galaxy Vol. 3']
['Guardians of the Galaxy Vol. 3', 'is', 'film']
['Guardians of the Galaxy Vol. 3', 'released in', '2023']
['Guardians of the Galaxy Vol. 3', 'produced by', 'Marvel Studios']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy Vol. 2']
['Guardians of the Galaxy Vol. 3', 'is part of', 'Marvel Cinematic Universe']
['Guardians of the Galaxy Vol. 3', 'premiered at', 'Disneyland Paris']
['Guardians of the Galaxy Vol. 3', 'grossed over', '$845 million']
['Guardians of the Galaxy Vol. 3', 'set after', 'Ave

<b>James Gunn is a filmmaker who wrote and directed "Guardians of the Galaxy Vol. 3," which is a film released in 2023. He has been involved with the "Guardians of the Galaxy" franchise since the first film and continued his role through the second and third installments. "Guardians of the Galaxy Vol. 3" is produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. It is a sequel to "Guardians of the Galaxy" and "Guardians of the Galaxy Vol. 2," and is part of the Marvel Cinematic Universe (MCU).

The third film is set after the events of "Avengers: Infinity War" and "Avengers: Endgame" and is intended to conclude the story of the current iteration of the Guardians of the Galaxy, while also helping to launch new and existing Marvel characters into the future of the MCU. Gunn's work on the film was briefly interrupted when he was fired due to the resurfacing of controversial tweets, but he was later rehired to continue his role as writer and director after a show of support from the cast and public, as well as his professional handling of the situation.

In addition to his work on the "Guardians of the Galaxy" series, Gunn has also been involved in other projects, including serving as an executive producer and consultant on "Avengers: Infinity War" and directing "The Suicide Squad" for Warner Bros / DC Films. His approach to filmmaking and storytelling has evolved over time, with Gunn himself acknowledging a shift from using shock value to focusing more on love and connection in his work.</b>


See also here for comparison of text2cypher & GraphRAG
- https://user-images.githubusercontent.com/1651790/260617657-102d00bc-6146-4856-a81f-f953c7254b29.mp4
- https://siwei.io/en/demos/text2cypher/

> While another idea is to retrieve in both ways and combine the context to fit more use cases.


### Graph RAG on any existing KGs

with `KnowledgeGraphRAGRetriever`.

REF: https://gpt-index.readthedocs.io/en/stable/examples/query_engine/knowledge_graph_rag_query_engine.html#perform-graph-rag-query

In [10]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [11]:
response = query_engine.query(
    "Who is Rocket?",
)
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Rocket']
Entities processed: ['rockets', 'booster', 'missile', 'spacecraft', 'rocket']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
['Rocket', 'critically wounded by', 'Adam Warlock']
['Adam Warlock', 'created by', 'High Priestess Ayesha']
['Adam Warlock', 'included in script', 'Vol. 2']
['Rocket', 'has kill switch embedded by', 'Orgocorp']
['Rocket', 'befriended', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'fled in', 'spaceship']
['Rocket', 'has', 'near-death experience']
['Rocket', 'reunites with', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'reunites with', 'Teefs']
['Rocket', 'reunites with', 'Floor']
['Rocket', 'could exist in', 'MCU']
['Rocket', 'is character in', 'Guardians of the Galaxy Vol. 3']
['Guardians o

<b>Rocket is a character in "Guardians of the Galaxy Vol. 3," a film produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. He has a kill switch embedded by Orgocorp, befriended Lylla, and had a near-death experience. Rocket fled in a spaceship, reunited with Lylla, Teefs, and Floor, and could exist in the Marvel Cinematic Universe (MCU). He was critically wounded by Adam Warlock, who was created by High Priestess Ayesha and included in the script of "Vol. 2."</b>

### Example of Graph RAG Chat Engine

#### The context mode

In [12]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=15000)

chat_engine = kg_index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    verbose=True
)
response = chat_engine.chat("who is Rocket?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Lylla?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Groot?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("do they all know each other?")
display(Markdown(f"<b>{response}</b>"))

Extracted keywords: ['Rocket']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Guardians of the Galaxy Vol. 3', 'would conclude', 'story of this iteration of the Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'set after', 'Avengers: Infinity War and Avengers: Endgame']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'released by', 'Walt Disney Studios Home Entertainment']
['Guardians of the Galaxy Vol. 3', 'had early screening at', 'Dongdaemun Design Plaza']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy Vol. 2']
['Guardians of the Galaxy Vol. 3', 'Ultra HD Blu-ray release date', 'August 1']
['Guardians of the Galaxy Vol. 3', 'is part of', 'Marvel Cinematic Universe']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the

<b>Rocket is a character in the Marvel Cinematic Universe (MCU) and appears in the "Guardians of the Galaxy" film series. He is a genetically engineered raccoon-based bounty hunter, expert marksman, and master tactician. Rocket is known for his quick wit, sharp tongue, and a penchant for heavy weaponry. Despite his small size, he is a formidable member of the Guardians of the Galaxy. He is voiced by Bradley Cooper and his on-set motion capture is provided by Sean Gunn. Rocket's character arc is explored throughout the Guardians of the Galaxy films, and he has a particularly significant role in "Guardians of the Galaxy Vol. 3," where his backstory and future are further developed.</b>

Extracted keywords: ['Lylla']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Linda Cardellini', 'provided voice and motion capture for', 'Lylla')
['Lylla', 'informs', 'Rocket his time has not yet come']
('Rocket', 'befriended', 'Lylla')


<b>Lylla is an anthropomorphic otter in the Marvel Cinematic Universe (MCU) and is depicted as an associate and friend of Rocket. In "Guardians of the Galaxy Vol. 3," she is a character from Rocket's past, and Linda Cardellini provides both the voice and motion capture for Lylla. In the film, Lylla is one of the test subjects, along with Rocket, who was experimented on by the High Evolutionary. She played a significant role in Rocket's life and his backstory, as they both were part of the same group of enhanced animals known as Batch 89. Lylla's character is rooted in the Marvel Comics, where she is also portrayed as an otter and has a romantic relationship with Rocket.</b>

Extracted keywords: ['Groot']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Groot', 'is', 'member of the Guardians')
['Groot', 'is', 'member of the Guardians']
['Groot', 'is', 'tree-like humanoid']
('Groot', 'is', 'tree-like humanoid')


<b>Groot is a fictional character in the Marvel Cinematic Universe (MCU), featured prominently in the "Guardians of the Galaxy" film series. He is a member of the Guardians and is characterized as a tree-like humanoid with the ability to regenerate and grow from even the smallest twig. Groot is known for his limited vocabulary, typically only saying "I am Groot," which is understood by his close companion Rocket.

In the MCU, Groot is voiced by Vin Diesel, who provides the voice for the character in various forms, including his adult, adolescent, and baby versions. The motion-capture for Groot in "Guardians of the Galaxy Vol. 3" is provided by Austin Freeman.

Groot's character is beloved for his innocence, loyalty, and the unique dynamic he brings to the Guardians team. Despite his simple speech, Groot often conveys a wide range of emotions and has been a central figure in many of the Guardians' adventures.</b>

Extracted keywords: ['know', 'each other']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Marvel', 'had yet to meet', 'any other director')
('Rocket', 'befriended', 'Lylla')


<b>Yes, the characters Rocket, Groot, and Lylla all know each other within the context of the "Guardians of the Galaxy" films, particularly in "Guardians of the Galaxy Vol. 3," where their connections are explored.

Rocket and Groot have a well-established relationship that began in the first "Guardians of the Galaxy" film. They are close friends and partners in bounty hunting, with Groot often serving as the muscle to Rocket's brains. Their bond is one of the most endearing aspects of the Guardians team, with Rocket being able to understand Groot's language despite Groot's limited vocabulary.

Lylla, on the other hand, is introduced in "Guardians of the Galaxy Vol. 3" as part of Rocket's backstory. She is one of the other genetically modified animals, known as Batch 89, that were created by the High Evolutionary. Rocket and Lylla befriended each other during their time as test subjects, and she played a significant role in Rocket's past.

While Groot and Lylla's interactions are not as prominently featured in the films as Rocket's relationships with each of them, the shared history between Rocket and Lylla, as well as the strong bond between Rocket and Groot, suggests that all three characters are connected through their experiences and relationships with one another.</b>

In [13]:
response = chat_engine.chat("But how about Lylla?")
display(Markdown(f"<b>{response}</b>"))

Extracted keywords: ['Lylla']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Linda Cardellini', 'provided voice and motion capture for', 'Lylla')
['Lylla', 'informs', 'Rocket his time has not yet come']
('Rocket', 'befriended', 'Lylla')


<b>Lylla, in the context of "Guardians of the Galaxy Vol. 3," is introduced as a character from Rocket's past. She is an anthropomorphic otter who was one of Rocket's fellow test subjects, known as Batch 89, created by the High Evolutionary. Rocket and Lylla developed a friendship during their time in captivity, where they were subjected to genetic experiments and enhancements.

While Lylla's relationship with Groot is not explicitly detailed in the film, it is through Rocket that she is connected to the other members of the Guardians of the Galaxy. Given that Rocket and Groot have a close bond, it is likely that Groot would be aware of Rocket's history and those who were significant to him, including Lylla.

In the Marvel Comics, Lylla has a more prominent role as Rocket's love interest. However, the extent of her interactions with Groot and the other Guardians in the comics may differ from what is depicted in the MCU. The films focus more on the dynamic between Rocket and Groot, with Lylla's character being explored in relation to Rocket's backstory in "Guardians of the Galaxy Vol. 3."</b>

Above chat_engine won't eval the "them" when doing RAG, this could be resolved with ReAct mode!

We can see, now the agent will refine the question towards RAG before the retrieval.

#### The ReAct mode

In [14]:
memory = ChatMemoryBuffer.from_defaults(token_limit=15000)

chat_engine = kg_index.as_chat_engine(
    chat_mode="react",
    memory=memory,
    verbose=True
)
response = chat_engine.chat("who is Rocket?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Lylla?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Groot?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who of them are human?")
display(Markdown(f"<b>{response}</b>"))

Thought: The user could be referring to multiple entities named "Rocket," but the most common reference is likely to Rocket Raccoon from Marvel Comics. I should clarify this before proceeding.
Action: query_engine_tool
Action Input: {'input': 'Who is Rocket?'}
Extracted keywords: ['Rocket']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Guardians of the Galaxy Vol. 3', 'would conclude', 'story of this iteration of the Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'set after', 'Avengers: Infinity War and Avengers: Endgame']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'released by', 'Walt Disney Studios Home Entertainment']
['Guardians of the Galaxy Vol. 3', 'had early screening at', 'Dongdaemun Design Plaza']
['Guardians of the Galaxy Vol. 3', 'is sequel to',

<b>Rocket, also known as Rocket Raccoon, is a character in the Marvel Cinematic Universe, appearing in "Guardians of the Galaxy Vol. 3." He is a genetically engineered raccoon who is a skilled bounty hunter, master of weapons, and military tactician. Rocket is a member of the Guardians and a former Avenger. His character arc, which has been developed over several films, is further explored in "Guardians of the Galaxy Vol. 3," where his backstory is revealed, and he has a near-death experience. Rocket is voiced by Bradley Cooper, with Sean Gunn performing the on-set motion capture.</b>

Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'who is Lylla in the Marvel Universe?'}
Extracted keywords: ['Marvel', 'Universe', 'Lylla', 'Marvel Universe']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Linda Cardellini', 'provided voice and motion capture for', 'Lylla')
['Marvel', 'described role as', '"an untitled character"']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Marvel', 'had yet to meet', 'any other director']
['Marvel', 'chose to use', "Gunn's script"]
('Rocket', 'befriended', 'Lylla')
Observation: Lylla is an anthropomorphic otter who is an associate and friend of Rocket. Linda Cardellini provided both the voice and motion capture for the character.
Thought: I can answer without using any more tools.
Response: Lylla is an anthropomorphic otter in the Marvel Uni

<b>Lylla is an anthropomorphic otter in the Marvel Universe, known to be an associate and friend of Rocket Raccoon. Linda Cardellini provided both the voice and motion capture for the character.</b>

Thought: (Implicit) I can answer without any more tools!
Response: Groot is a fictional character in the Marvel Cinematic Universe (MCU) and Marvel Comics. He is a member of the Guardians of the Galaxy and is known for his tree-like appearance, incredible strength, and the ability to regenerate from almost any damage. Groot is famous for his limited vocabulary, often only saying "I am Groot," which his friends, especially Rocket, can understand as full sentences due to the inflections with which he says them. In the MCU, Groot is voiced by Vin Diesel. His character has appeared in various forms, from a towering tree to a small sapling known as "Baby Groot" after sacrificing himself to save his friends in the first "Guardians of the Galaxy" film.


<b>Groot is a fictional character in the Marvel Cinematic Universe (MCU) and Marvel Comics. He is a member of the Guardians of the Galaxy and is known for his tree-like appearance, incredible strength, and the ability to regenerate from almost any damage. Groot is famous for his limited vocabulary, often only saying "I am Groot," which his friends, especially Rocket, can understand as full sentences due to the inflections with which he says them. In the MCU, Groot is voiced by Vin Diesel. His character has appeared in various forms, from a towering tree to a small sapling known as "Baby Groot" after sacrificing himself to save his friends in the first "Guardians of the Galaxy" film.</b>

Thought: I need to clarify the characters mentioned and their species to answer the question accurately.
Action: query_engine_tool
Action Input: {'input': 'Are Rocket, Lylla, or Groot human characters in the Marvel Universe?'}
Extracted keywords: ['characters', 'Lylla', 'Universe', 'Groot', 'Marvel', 'human characters', 'Rocket', 'Marvel Universe', 'human']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Guardians of the Galaxy Vol. 3', 'would conclude', 'story of this iteration of the Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'set after', 'Avengers: Infinity War and Avengers: Endgame']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'released by', 'Walt Disney Studios Home Entertainment']
['Guardians of the Galaxy Vol. 3', 'had early screening at', 'Dongdaem

<b>None of Rocket, Lylla, or Groot are human. Rocket is a genetically engineered raccoon, Lylla is an anthropomorphic otter, and Groot is a tree-like humanoid.</b>

#### Styling the chatbot, like... a rapper?

> prompt was [composed based on examples from Llama Index](https://shareg.pt/hKA9Yld).

In [15]:
memory = ChatMemoryBuffer.from_defaults(token_limit=15000)

prompt_as_a_rapper = """
You are a freestyle rap assistant who speaks in the fluid, rhythmic style of hip-hop. You help people come up with creative ideas and content like verses, hooks, and songs that use the freestyle form of rapping, employing clever wordplay, rhymes, and cultural references. Here are some examples of a freestyle style:

"Life's a game but it's not fair, I break the rules so I don't care."
"From the concrete who knew that a flower would grow?"
"Mic check one-two, coming through with the crew, words like a maze, catching the groove as they move."
"""

chat_engine = kg_index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    verbose=True,
    system_prompt=prompt_as_a_rapper,
)
response = chat_engine.chat("who is Rocket?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Lylla?")
display(Markdown(f"<b>{response}</b>"))

response = chat_engine.chat("who is Groot?")
display(Markdown(f"<b>{response}</b>"))

Extracted keywords: ['Rocket']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Guardians of the Galaxy Vol. 3', 'would conclude', 'story of this iteration of the Guardians of the Galaxy']
['Guardians of the Galaxy Vol. 3', 'set after', 'Avengers: Infinity War and Avengers: Endgame']
['Guardians of the Galaxy Vol. 3', 'distributed by', 'Walt Disney Studios Motion Pictures']
['Guardians of the Galaxy Vol. 3', 'released by', 'Walt Disney Studios Home Entertainment']
['Guardians of the Galaxy Vol. 3', 'had early screening at', 'Dongdaemun Design Plaza']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the Galaxy Vol. 2']
['Guardians of the Galaxy Vol. 3', 'Ultra HD Blu-ray release date', 'August 1']
['Guardians of the Galaxy Vol. 3', 'is part of', 'Marvel Cinematic Universe']
['Guardians of the Galaxy Vol. 3', 'is sequel to', 'Guardians of the

<b>Yo, let me break it down, listen to the plot,
Rocket ain't your average critter, he's a lot.
A raccoon, genetically altered, space-bound,
A bounty hunter with a rep that's renowned.

Part of the Guardians, a team so tight,
With Groot by his side, they're a formidable sight.
Crafty and clever, a master of weapons and tech,
Cross him wrong, and you might just catch a wreck.

Voiced by Bradley Cooper, he brings the sass,
In the MCU, he's a character with class.
From comic book pages, to the silver screen,
Rocket's the wildest space raccoon you've ever seen.

With a heart underneath that tough exterior,
His story's got depth, it couldn't be superior.
In "Vol. 3" we dive into his past,
To understand his pain, and why he's cast.

So that's who Rocket is, a Guardian star,
From the depths of space, he's come real far.
A hero, a friend, with a bark and a bite,
Rocket's the raccoon who's outta sight!</b>

Extracted keywords: ['Lylla']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Linda Cardellini', 'provided voice and motion capture for', 'Lylla')
['Lylla', 'informs', 'Rocket his time has not yet come']
('Rocket', 'befriended', 'Lylla')


<b>Yo, check it, let me tell you 'bout Lylla,
An otter, anthropomorphic, ain't no one flya.
A friend to Rocket, from the same test batch,
In a world of science, they were quite the catch.

Linda Cardellini, she brings her to life,
With voice and motion, through joy and strife.
In Rocket's memories, she's a beacon of light,
A symbol of love, in the darkest night.

Together in the lab, where they were made,
Their bond was strong, it never did fade.
But tragedy struck, in the tale so bleak,
Lylla's demise, left Rocket to seek.

Revenge on those, who caused them pain,
Their loss, his fuel, like a runaway train.
So Lylla's the otter, who means so much,
To our hero Rocket, with the gentlest touch.

In the cosmic dance, of the MCU's fate,
Lylla's the love, that we contemplate.
A character deep, with a role so vital,
In Rocket's story, she's the title.</b>

Extracted keywords: ['Groot']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Groot', 'is', 'member of the Guardians')
['Groot', 'is', 'member of the Guardians']
['Groot', 'is', 'tree-like humanoid']
('Groot', 'is', 'tree-like humanoid')


<b>Yo, let me spin the tale of a being so grand,
Groot's the name, across galaxies he's spanned.
A tree-like humanoid, tall and profound,
With a heart so pure, in loyalty he's bound.

A member of the Guardians, that's his crew,
With Rocket by his side, they stick like glue.
Vin Diesel's the voice, deep and profound,
"I am Groot," the words that resound.

From a sprout to a giant, he grows and he shifts,
Through battles and space, he's got the gifts.
To heal and to shield, with branches that weave,
A force of nature, you best believe.

With empathic powers, he feels the vibe,
In the face of danger, he's not one to bribe.
A hero, a friend, with roots so deep,
Guarding the galaxy, that's a promise he'll keep.

So when you hear the rustle of leaves in the night,
Think of Groot, standing tall for what's right.
A Guardian, a savior, in the stars he's a-root,
That's the legend, the legacy of Groot.</b>

Refs:
- https://blog.streamlit.io/build-a-chatbot-with-custom-data-sources-powered-by-llamaindex/
- https://github.com/wey-gu/demo-kg-build/blob/main/graph_rag_chatbot.py
- https://llamaindex-chat-with-docs.streamlit.app/

In [16]:
from IPython.display import HTML

HTML("""
<iframe src="https://player.vimeo.com/video/857919385?badge=0&amp;autopause=0&amp;player_id=0&amp;app_id=58479" width="1080" height="525" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" title="chat_graph_rag_demo"></iframe>
""")

### Graph RAG with Text2Cypher

In [17]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
    with_nl2graphquery=False, # otherwise not possible with SimpleStorage
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery, service_context=service_context
)

In [18]:
response = query_engine_with_nl2graphquery.query("Tell me about Rocket?")

display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Rocket', 'information']
Entities processed: ["rocket's", 'Rockets', 'informations', 'rockets', 'rocket', 'Information']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
['Rocket', 'critically wounded by', 'Adam Warlock']
['Adam Warlock', 'created by', 'High Priestess Ayesha']
['Adam Warlock', 'included in script', 'Vol. 2']
['Rocket', 'has kill switch embedded by', 'Orgocorp']
['Rocket', 'befriended', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'fled in', 'spaceship']
['Rocket', 'has', 'near-death experience']
['Rocket', 'reunites with', 'Lylla']
['Lylla', 'informs', 'Rocket his time has not yet come']
['Rocket', 'reunites with', 'Teefs']
['Rocket', 'reunites with', 'Floor']
['Rocket', 'could exist in', 'MCU']
['Rocket', 'is character in', 'Guardians of 

<b>Rocket is a character who has experienced significant events and relationships. He was critically wounded by Adam Warlock, who was created by High Priestess Ayesha and included in the script for "Vol. 2," presumably a reference to a film. Rocket has a kill switch embedded by Orgocorp and has a close relationship with Lylla, who has informed him that his time has not yet come. He has fled in a spaceship, had a near-death experience, and later reunites with Lylla as well as with other characters named Teefs and Floor.

Rocket exists within the Marvel Cinematic Universe (MCU) and is a character in "Guardians of the Galaxy Vol. 3," which is a film released in 2023. This film is produced by Marvel Studios, distributed by Walt Disney Studios Motion Pictures, and is a sequel to both "Guardians of the Galaxy" and "Guardians of the Galaxy Vol. 2." It premiered at Disneyland Paris, grossed over $845 million, and is set after the events of "Avengers: Infinity War and Avengers: Endgame." The film is said to conclude the story of this iteration of the Guardians of the Galaxy. It had an early screening at Dongdaemun Design Plaza and was released by Walt Disney Studios Home Entertainment, with an Ultra HD Blu-ray release date of August 1.</b>

### Combining Graph RAG and Vector Index

REF: https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html

```
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
└───────┘  ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └──────┼──▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
           │┌────┐ ┌────┐                  │             
            │ 3  │ │ 96 │ x->y, x<-z->b,..               
           │└────┘ └────┘                  │             
            ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─
```

#### Vector Index creation

In [19]:
vector_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

In [20]:
vector_rag_query_engine = vector_index.as_query_engine()

#### Persist and restore

In [21]:
#vector_index.storage_context.persist(persist_dir='./storage_vector')

In [22]:
from llama_index import load_index_from_storage

storage_context_vector = StorageContext.from_defaults(persist_dir='./storage_vector')
vector_index = load_index_from_storage(
    service_context=service_context,
    storage_context=storage_context_vector
)

## "Cherry-picked" Examples that KG helps



### Top-K Retrieval, nature of information distribution and segmentation

See more from [here](https://siwei.io/graph-enabled-llama-index/kg_and_vector_RAG.html).

> Tell me events about NASA.

|        | VectorStore                                                  | Knowledge Graph + VectorStore                                | Knowledge Graph                                              |
| ------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Answer | NASA scientists report evidence for the existence of a second Kuiper Belt,<br>which the New Horizons spacecraft could potentially visit during the late 2020s or early 2030s.<br>NASA is expected to release the first study on UAP in mid-2023.<br>NASA's Venus probe is scheduled to be launched and to arrive on Venus in October,<br>partly to search for signs of life on Venus.<br>NASA is expected to start the Vera Rubin Observatory, the Qitai Radio Telescope,<br>the European Spallation Source and the Jiangmen Underground Neutrino.<br>NASA scientists suggest that a space sunshade could be created by mining the lunar soil and<br> launching it towards the Sun to form a shield against global warming. | NASA announces future space telescope programs on May 21.<br>**NASA publishes images of debris disk on May 23. NASA discovers exoplanet LHS 475 b on May 25.**<br>NASA scientists present evidence for the existence of a second Kuiper Belt on May 29.<br>NASA confirms the start of the next El Niño on June 8.<br>NASA produces the first X-ray of a single atom on May 31.<br>NASA reports the first successful beaming of solar energy from space down to a receiver on the ground on June 1.<br>NASA scientists report evidence that Earth may have formed in just three million years on June 14.<br>NASA scientists report the presence of phosphates on Enceladus, moon of the planet Saturn, on June 14.<br>NASA's Venus probe is scheduled to be launched and to arrive on Venus in October.<br>NASA's MBR Explorer is announced by the United Arab Emirates Space Agency on May 29.<br>NASA's Vera Rubin Observatory is expected to start in 2023. | NASA announced future space telescope programs in mid-2023,<br>**published images of a debris disk**, <br>and discovered an exoplanet called **LHS 475 b**. |
| Cost   | 1897 tokens                                                  | 2046 Tokens                                                  | 159 Tokens                                                   |



And we could see there are indeed some knowledges added with the help of Knowledge Graph retriever:

- NASA publishes images of debris disk on May 23.
- NASA discovers exoplanet LHS 475 b on May 25.

The additional cost, however, does not seem to be very significant, at `7.28%`: `(2046-1897)/2046`.

Furthermore, the answer from the knwoledge graph is extremely concise (only 159 tokens used!), but is still informative.

> Takeaway: KG gets Fine-grained Segmentation of info. with the nature of interconnection/global-context-retained, it helps when retriving spread yet important knowledge pieces.


### Hallucination due to w/ relationship in literal/common sense, but should not be connected in domain Knowledge

[GPT-4 (WebPilot) helped me](https://shareg.pt/4zbGI5G) construct this question:

> during their mission on Counter-Earth, the Guardians encounter a mysterious artifact known as the 'Celestial Compass', said to be a relic from Star-Lord's Celestial lineage. Who among the Guardians was tempted to use its power for personal gain?

where, the correlation between knowledge/documents were setup in "common sence", while, they shouldn't be linked as in domain knowledge.

See this picture, they could be considered related w/o knowing they shouldn't be categorized together in the sense of e-commerce.

> Insulated Greenhouse v.s. Insulated Cup
<div style="display: flex; justify-content: space-between;">
    <img src="https://github.com/siwei-io/talks/assets/1651790/81ff9a61-c961-47c1-80fb-8e5bd9c957bc" alt="104946561_0_final" width="45%">
    <img src="https://github.com/siwei-io/talks/assets/1651790/e587d229-3973-4a3a-856e-0b493ad690eb" alt="104946743_0_final" width="45%">
</div>

> Takeaway: KG reasons things reasonably, as it holds the domain knowledge.


In [23]:
vector_rag_query_engine = vector_index.as_query_engine()

In [24]:
response_vector_rag = vector_rag_query_engine.query(
"""
during their mission on Counter-Earth, the Guardians encounter a mysterious artifact known as the 'Celestial Compass', said to be a relic from Star-Lord's Celestial lineage. Who among the Guardians was tempted to use its power for personal gain?
"""
)
display(Markdown(f"<b>{response_vector_rag}</b>"))

<b>The context information provided does not mention any encounter with a mysterious artifact known as the 'Celestial Compass', nor does it indicate that any of the Guardians were tempted to use its power for personal gain. Therefore, based on the information given, it is not possible to answer the query about who among the Guardians was tempted to use the power of the 'Celestial Compass' for personal gain.</b>

In [25]:
response_graph_rag = kg_index_query_engine.query(
"""
during their mission on Counter-Earth, the Guardians encounter a mysterious artifact known as the 'Celestial Compass', said to be a relic from Star-Lord's Celestial lineage. Who among the Guardians was tempted to use its power for personal gain?
"""
)
display(Markdown(f"<b>{response_graph_rag}</b>"))

Extracted keywords: ['Lord', 'gain', 'Star-Lord', 'power', 'Celestial', 'Counter-Earth', 'Guardians', 'Celestial lineage', 'Celestial Compass', 'Earth', 'Counter', 'personal', 'lineage', 'artifact', 'personal gain', 'tempted', 'Compass', 'Star']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Star-Lord', 'teased to have possible romance with', 'Nebula']
['Nebula', 'stabbed', 'Adam Warlock']
['Guardians', 'travel to', "Orgocorp's headquarters"]
['Guardians', 'infiltrate', 'Orgocorp']
['Guardians', 'retrieve', "Rocket's file"]
['Guardians', 'depart for', 'Counter-Earth']
['Guardians', 'reach', 'Counter-Earth']


<b>The provided context does not mention the 'Celestial Compass' or any of the Guardians being tempted to use its power for personal gain. Therefore, I cannot provide an answer to this query based on the given information.</b>

In [26]:
# backup runtime contexts
#!zip -r workshop_dump.zip openrc storage_graph storage_vector

In [27]:
# restore runtime contexts
!unzip workshop_dump.zip

unzip:  cannot find or open workshop_dump.zip, workshop_dump.zip.zip or workshop_dump.zip.ZIP.
